# Jugando Atari con Julia y Reinforcement Learning

No nos emocionemos aun, todavia hay un largo camino por recorrer... Entrenar un programa (agente) que aprenda a jugar a nivel humano toma mucha paciencia y horas de entrenamiento; un solo paso equivocado y no lo vamos a lograr. 

Para aprender a resolver problemas complejos, lo primero que tenemos que hacer es comprender las complicaciones detras de cada parte de esta tarea. Para esto, necesitamos comenzar con problemas sencillos y escalarlos poco a poco. Necesitamos comenzar haciendo un pequeño viaje en el tiempo a la historia reciente de la Inteligencia Artificial y el Reinforcement Learning.

## 1. MDPs

Nuestra historia comienza con Richard Bellman (*A Markovian Decission Process* 1957) y Ronald Howard (*Dynamic Programming and Markov Processes* 1960), que extendio la teoria de Bellman.

 <center><table><tr>
  <td><img src="Images/bellman.jpg" alt="Richard Bellman" width="150">
  <figcaption>Fig1.a - El es Bellman.</figcaption></td>
  <td><img src="Images/howard.jpg" alt="Ronald Markov" width="150">
  <figcaption>Fig1.b - El es Howard.</figcaption></td>
  <td><img src="Images/markov.jpg" alt="Andrey Markov" width="150">
  <figcaption>Fig1.c - El es Markov.</figcaption></td>
</table></center>

Bellman tuvo una idea muy simple de como modelar un proceso de aprendizaje. En cada momento del tiempo $t$, un agente debe tomar una decision (accion) $A_t$ dependiendo del estado del ambiente $S_t$; en el momento que el agente toma su decision, interactua con el ambiente y lo transforma, llevando al sistema a un nuevo estado $S_{t+1}$. ¿Como puede saber el agente si actuo correcta o incorrectamente? Para eso necesita una señal de reforzamiento o pago. Por esta razon, suponemos ademas que el agente recibe un pago $R_{t+1}$ en el momento que el sistema se transforma.

 <center>
 <img src="Images/mdp_process.png" alt="MDP Representation" width="400" height="225">
 <figcaption>Fig2. - Este es el ciclo de una tarea de aprendizaje.</figcaption>
</center>

¿Y que tiene que ver con Markov? Bellman es uno de los desarrolladores de la *Programacion Dinamica*, que es una tecnica para resolver problemas matematicos en los que las variables que queremos encontrar dependen del tiempo y la solucion en un momento $t$ se puede expresar en terminos de la solucion al tiempo $t-1$. La idea central del trabajo de Bellman consiste en que para encontrar las decisiones optimas $A_t, A_{t+1},...$ de forma que maximicen el total de pagos recibidos en el tiempo $R_{t+1}, R_{t+2}, ...$ tenemos que suponer que el estado futuro del sistema es independiente del pasado condicionado a su estado actual y de las decisiones del agente. Esta propiedad se conoce como la propiedad de Markov. En terminos matematicos, la implicacion de esta condicion es
$$
P(S_{t+1}, R_{t+1} \mid S_0, A_0, R_0, ..., S_t, A_t, R_t) = P(S_{t+1}, R_{t+1} \mid S_t, A_t).
$$
En otras palabras, las probabilidades de transicion $(S_t,A_t) \to (S_{t+1}, R_{t+1})$ y las decisiones del agente determinan todo el comportamiento del sistema. Un problema reforzamiento con la propiedad de Markov se conoce como un *Markov Decission Process* o MDP.

## Un juego de Atari como un MDP

<center>
 <img src="Images/pong.jpg" alt="Pong" width="150">
 <figcaption>Fig3. - El juego de de Atari Pong.</figcaption>
</center>

Este tutorial es acerca de como resolver juegos de Atari en Julia. Para promover el desarollo de algoritmos de Inteligencia Artificial y Reinforcement Learning, la organizacion OpenAI creo la interfaz [Open AI Gym](https://gym.openai.com/) que facilita el desarollo y evaluacion de algoritmos de aprendizaje. La mala noticia es que solo esta disponible para Python y es casi imposible de instalar en Windows. La buena noticia es que Julia tiene una interfaz para interactuar con Python de forma muy sencilla y no sera una limitacion.

Por supuesto que pueden optar por implementar sus algoritmos directamente en Python y no en Julia. Pero este es un tutorial de Julia, que explota la belleza, facilidad y velocidad de Julia.

Por favor vean la pagina de Open AI Gym directamente para instrucciones de como instalar la libreria `gym` de Python. Para poder usar `gym` en Julia, deberan antes tener la paqueteria de Interfaz con Python llamada `PyCall`. Para instalarla, usen el comando `Pkg.add("PyCall")`.

El siguiente codigo muestra como cargar el juego de Pong y hacer unas cuantas simulaciones.

In [1]:
using PyCall # use Pkg.add("PyCall") to install
@pyimport gym # most have gym installed on your computer's Python distribution
pong = gym.make("Pong-v0") # crea el ambiente de juego Pong en el objeto pong

[2017-06-02 22:00:13,083] Making new env: Pong-v0


PyObject <TimeLimit<AtariEnv<Pong-v0>>>

Un MDP tiene estados y acciones disponibles, para saber cuales son con un objeto del tipo Pong, podemos usar las acciones de la paqueteria de Python gym.

### Conjunto de Estados

In [2]:
pong[:observation_space] # pong.observation_space en Python

PyObject Box(210, 160, 3)

El resultado es que el espacio de estados es una "caja" de dimensiones $210 \times 160 \times 3$. Aunque en los metodos de Reinforcement Learning no necesitaremos saber el significado de los estados,  en este caso sabemos de antemano que va el juego va a regresar los componentes RGB de los pixeles del juego. El juego tiene una resolucion de $210 \times 160$ y por cada uno tiene tres componentes de color. Un componente de color es un valor en el rango $0--255$. Si no supieramos esto, pudiese haber usado las siguientes funciones para determinar la frontera del espacio de observaciones
* `pong[:observation_space][:high]`
* `pong[:observation_space][:low]`

No las incluimos aqui porque el resultado es muy grande.

### Conjunto de Acciones

Determinar el conjunto de acciones es muy similar, para eso podemos usar

In [3]:
pong[:action_space] # pong.action_space en Python

PyObject Discrete(6)

En este caso es discreto (el conjunto de acciones siempre es discreto en estas pruebas). Eso quiere decir que solo puede obtener los valores $0-5$. Si! Empieza en cero, es una de las herencias por la interfaz con Python.

Otra manera de encontrar el numero de acciones es simplemente con el comando

In [4]:
pong[:action_space][:n] # pong.action_space en Python

6

Tanto el espacio de observacion y espacio de accion son objetos de Python que pertenecen a la clase "espacios" y tienen implementado un metodo muy util para tomas muestras aleatorias de estos espacios. Eso sera muy util al principio, para poder demostrar algunas simulaciones.

In [5]:
pong[:action_space][:sample]()

4

Afortunadamente para los metodos de Reinforcement Learning, tampoco necesitamos saber el significado de estas acciones.

### Simulando transiciones del MDP

Ahora lo mas divertido, veamos primero como simular una transicion usando el metodo `step` de los ambientes de `gym`.

In [6]:
initial_state = pong[:reset]() # comienza oficialmente el ambiente Pong poniendo en su estado inicial
action = pong[:action_space][:sample]() # escoje una accion al azar, ojo no depende del estado inicial
    # por ser completamente aleatorio, pero en principio deberia...
new_state, reward, done, info = pong[:step](action)
; # para que no imprima el resultado de la celda

El metodo `step` recibe una accion (en realidad, tambien recibe el estado actual, para ya esta codificado dentro del ambiente) y regresa cuatro cosas
* `new_state`: El nuevo estado del ambiente
* `reward` El pago recibido por la accion decididad
* `done` Una variable booleana que determina si el episodio esta terminado 
* `info` Un diccionario que trae informacion adicional del juego, en teoria nunca deberiamos necesitarla.

La funcion `step` tiene implica las probabilidades de transicion $(S_t, A_t)\to (S_{t+1}, R_{t+1})$ del MDP. No son explicitas porque no regresa toda la distribucion, simplemente un muestreo de esa distribucion. Esta es la dificultad a la que nos enfrentamos.

### Simulando y visualizando todo un episodio 

Veamos ahora como podriamos simular y visualizar un episodio entero. Para visualizar el estado del ambiente necesitaremos el metodo `render`. 

In [7]:
initial_state = pong[:reset]() 
done = false
episode_reward = 0.
while !done # cicla hasta que la funcion step marque el fin del episodio
    pong[:render]() # crea la visualizacion del juego
    action = pong[:action_space][:sample]() 
    state, reward, done, info = pong[:step](action)
    episode_reward += reward
end
println("Episodio terminado con pago total: ", episode_reward)
pong[:render](close = true) # cierra la visualizacion del juego 

Episodio terminado con pago total: -

**Felicidades!!!** Con esta parte del tutorial, ya sabes todo lo que necesitabas de los ambientes de `gym`! 

Desafortunadamente, resolver el juego de Pong no es facil pues el espacio de estados viene en terminos de pixeles y eso significa muchas posibilidades (aun cuando no las veamos todas). De momento, trataremos de resolver un problema mas sencillo, primero deberan hacer el siguiente ejercicio.

### <span style='color: red'> Ejercicio 1 </span>

Antes de poder juegos de Atari, vamos a utilizar el ambiente "MountainCar-v0" que es una muy sencilla y clasica prueba de algoritmos de Inteligencia Artificial. El problema consiste en un coche que tiene que subir una colina pero su motor no tiene la fuerza necesaria. Para esto tiene que ir de lado en lado construyendo inercia.

<center>
 <img src="Images/mountainCar.jpg" alt="Mountain Car" width="300" height="225">
 <figcaption>Fig3. - Problema del coche de la colina.</figcaption>
</center>
 
Introduce el codigo para cargar el ambiente "MountainCar-v0" y realiza la simulacion de 10 episodios tomando decisiones completamente aleatorias con la funcion `sample` del espacion del ambiente. Tu codigo debe imprimir 
1. El pago total acumulado por cada episodio
2. El total de pasos por episodio
3. El tiempo total por episodio, puedes usar las funciones tic() y toq() de julia para esto.



In [8]:
using PyCall # use Pkg.add("PyCall") to install
@pyimport gym # most have gym installed on your computer's Python distribution
hill = gym.make("MountainCar-v0") # crea el ambiente de juego Pong en el objeto pong

[2017-06-02 22:00:22,763] Making new env: MountainCar-v0


PyObject <TimeLimit<MountainCarEnv<MountainCar-v0>>>

In [9]:
initial_state = hill[:reset]() 
done = false
episode_reward = 0.
step = 0.
time = 0.

for i = 0:9
    initial_state = hill[:reset]() 
    done = false
    episode_reward = 0.
    step = 0.
    time = 0.
    tic()
  while !done # cicla hasta que la funcion step marque el fin del episodio
    timeIni=tic()
    hill[:render]() # crea la visualizacion del juego
    action = hill[:action_space][:sample]() 
    state, reward, done, info = hill[:step](action)
    episode_reward += reward
    step = step + 1
    time += toc()    
end

    
println("Episodio terminado con pago total: ", episode_reward)
println("El total de pasos por episodio: ", step)
println("El tiempo total por episodio: ", time)
hill[:render](close = true) # cierra la visualizacion del juego 
end



elapsed time: 0.130125864 seconds
elapsed time: 0.0084982 seconds
elapsed time: 0.002813525 seconds
elapsed time: 0.002954883 seconds
elapsed time: 0.010196327 seconds
elapsed time: 0.002643373 seconds
elapsed time: 0.003348952 seconds
elapsed time: 0.010406166 seconds
elapsed time: 0.002733534 seconds
elapsed time: 0.009197843 seconds
elapsed time: 0.006241043 seconds
elapsed time: 0.003163309 seconds
elapsed time: 0.007058781 seconds
elapsed time: 0.006459401 seconds
elapsed time: 0.013271037 seconds
elapsed time: 0.014305299 seconds
elapsed time: 0.004106208 seconds
elapsed time: 0.007351218 seconds
elapsed time: 0.010416408 seconds
elapsed time: 0.012474625 seconds
elapsed time: 0.00491449 seconds
elapsed time: 0.012015625 seconds
elapsed time: 0.006695761 seconds
elapsed time: 0.009346241 seconds
elapsed time: 0.007732926 seconds
elapsed time: 0.005012532 seconds
elapsed time: 0.0082919 seconds
elapsed time: 0.004787378 seconds
elapsed time: 0.014668557 seconds
elapsed time: 0.005

# 2. Control optimo

La regla de toma de decision del agente se conoce como *politica*. La regla no tiene porque ser deterministica, y de hecho, matematicamente conviente pensar esta regla como una distribucion de probabilidad sobre todas las posibles acciones. Para ser mas preciso, una politica es una forma de convertir el estado actual $S_t$ en un accion $A_t$, es decir, es una familia de distribuciones condicionales $\pi(A_t \mid S_t) := P(A_t \mid S_t)$.

El problema del control optimo es el de encontrar una politica $\pi^*$ para un MDP que maximice la ganancia total 
$$
\pi^* = \mathrm{argmax}_\pi E \left[ R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} \cdots  \gamma^{T-1} R_T\right].
$$
En esta formula, necesitamos una esperanza porque la suma total de ganancias es aleatoria. El simbolo $\gamma$ es una constante $0<\gamma\leq 1$ que representa la preferencia intertemporal. Cuando $\gamma < 1$, se valora mas un retorno en el presente que en el futuro; nosotros supondremos $\gamma = 1$ por simplicidad.

Teoricamente, el trabajo de Bellman y Howard de Programacion Dinamica para MDPs mostro que siempre es posible encontrar la politica optima si se conoce completamente las probabilidades de transicion $P(S_{t+1}, R_{t+1}| S_t, A_t)$ del MDP. En la practica, enfrentamos un problema adicional: normalmente el agente intentando aprender no conoce las probabilidades de transicion del MDP y por lo tanto no puede recurrir a los metodos exactos de la programacion dinamica para encontrar una politica optima; mas aun, aun cuando si pueda conocer estas probabilidades, en ocasiones la cantidad de estados del sistema es tan grande que los metodos analiticos de la programacion dinamica no son suficiente (por ejemplo, se estima que una partida de ajedrez puede tener hasta $10^{45}$ configuraciones distintas, totalmente intratable para los metodos exactos).


## Las Ecuaciones de Bellman

Aunque los metodos de Programacion Dinamica son frecuentemente impracticos para buscar politicas optimas, son la piedra angular para entender las estrategias exitosas de solucion. Si un sistema se encuentra en un estado $S_t$, el valor esperado que recibira el agente al tomar la decision $A_t$ es
$$
Q_\pi(S_t,A_t) := E_\pi\left[ R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} \cdots  \gamma^{T-1} R_T \mid S_t, A_t \right]
$$
Las ecuaciones de Bellman son ecuaciones recursivas en terminos de la *la funcion de valor estado-accion $Q$*. La propiedad de Markov garantiza que en cualquier momento del tiempo, el futuro del sistema depende solo de su estado actual y de la decision del agente. Como consecuencia de esta "perdida de memoria", despues de una transicion $(S_t, A_t) \to (S_{t+1}, R_{t+1})$ y una proxima accion $A_{t+1}$ del agente, el sistema vuelve a empezar, por lo que se tiene la identidad
$$
Q_\pi(S_t,A_t) = E_\pi\left[R_{t+1} + \gamma Q_\pi(S_{t+1}, A_{t+1})]\right] \quad\quad \text{(Ecuacion de Bellman 1)}
$$
La ecuacion anterior debe una historia:

<blockquote> El agente (con politica) $\pi$ se encuentra en el estado $S_t$ y decide la accion $A_t$. Justo en este momento tiene una promesa de pago futuro de $Q_\pi(S_t, A_t)$. Tras decidir, el sistema transiciona al estado $S_{t+1}$ recibiendo un pago recibe un pago de $R_{t+1}$ por su decision. Ahora tendra que decidir nuevamente una accion $A_{t+1}$ y tendra una promesa de pago futuro de $Q_\pi(S_{t+1}, A_{t+1})$. El pago $R_{t+1}$ que recibio mas la esperanza de pago futuro $Q_\pi(S_{t+1}, A_{t+1})$ deben de ser igual a la promesa de pago $Q_\pi(S_t, A_t)$ que tenia entes de la transicion.
</blockquote>

Esta ecuacion es la primera de las ecuaciones de Bellman y se cumple para todas las politicas $\pi$. La utilidad de esta ecuacion es evaluar la politica actual $\pi$. 

Aunque existen formar de mejorar la politica $\pi$ una vez que se conoce $Q_\pi$, Bellman tambien encontro una ecuacion recursiva que se cumple unicamente por la politica optima $\pi^*$ y que puede ser usada para encontrar la politica optima directamente. La segunda ecuacion o *ecuacion de optimalidad de Bellman* dice

$$
Q_{\pi^*}(S_t,A_t) = E_{\pi^*}\left[ R_{t+1} + \gamma \max_A Q_{\pi^*}(S_{t+1}, A)]\right] \quad\quad \text{(Ecuacion de Bellman 2)}.
$$

El mecanismo detras de esta expresion recursiva descubierta por Bellman se basa en que un individuo racional siempre escoge la accion que maximiza su utilidad esperada, que en terminos de la funcion $Q_{\pi^*}$ implica
$$
\pi^*(S_t) = \mathrm{argmax}_A Q_{\pi^*}(S_t,A).
$$


## Reinforcement Learning

El *Reinforcement Learning* (RL) busca resolver las limitaciones que de la programacion Dinamica tradicional para resolver MDPs. En RL tener un modelo del ambiente (probabilidades de transicion) es optativo y sirve para hacer planeacion y mejorar los algoritmos, pero no es necesario. Las estrategias de Reinforcement Learning evaluan una politica y la mejoran siguiendo estrategias iterativas. 

Podriamos agrupar a los diversos metodos del RL en tres grupos dependiendo de su grado de "supervision":

1. **Metodos de Programacion Dinamica**: Utilizan de forma directa las ecuaciones de Bellman y necesitan de un conocimiento de las probabilidades de transicion $P(S_{t+1},A_{t+1}\mid S_t, A_t)$, de lo contrario, pueden intentar estimarlas estadisticamente a traves de simulacion. Aunque son muy precisos, son altamente demandantes en terminos de memoria y poder computacional, y requieren de un conocimiento de las probabilidades de transicion del ambiente que rara vez se tienen completamente disponibles.
2. **Metodos de Funciones de Valor**: En vez de intentar conocer las probabilidades de transicion, estos metodos buscan estimar la funcion de valor $Q$ de la politica optima usando la ley de los grandes numeros (LGN). Una vez estimada la funcion Q optima, podemos recuperar la politica optima poniendo $\pi^*(S_t) = \mathrm{argmax}_A Q_{\pi^*}(S_t,A)$. Mas adelante veremos en que formas las Ecuaciones de Bellman y la LGN permiten conocer $Q$.
3. **Metodos de Gradientes de Politica**: Este ultimo grupo es el menos supervisado y trata directamente  de encontrar unas distribuciones de probabilidad $\pi(A \ S)$ para cada pareja estado-accion $(S,A)$, sin pasar por la funcion $Q$, y tratando de maximizar la utilidad acumulada $R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{T-1}$. Es es el metodo buscar patrones en los estados $S_t$ que directamente se correlacionen con mayores pagos.

Ambos enfoque 2 y 3 son muy populares actualmente. La ventaja de los ultimos es que tienen menos supuestos al ser menos estructrados. Su desventaja es que tambien son mayores cajas negras y en la practica pueden ser tan o mas dificiles de implementar. Aun no queda claro cual es el mejor enfoque, si es que hay alguno, pueden leer una comparacion en [este link](http://www.mcgovern-fagg.org/amy/pubs/tr-06-001.pdf).

**Nosotros vamos a utilizar el el enfoque de funciones de valor**, pues da un equilibrio entre estructura y practicidad.

## Politcas optimas y $\epsilon$-greegy 

En la realidad, nuestros algoritmos no escogeran siempre la decision optima $\pi^*(S_t)$, pero si con una probabilidad $(1-\epsilon)$ donde $\epsilon$ ira decreciendo con el tiempo. El problema de escoger siempre la decision optima es que en la practica no sabemos a la perfeccion los parametros del MDP, y un agente que no explore, tiene el riesgo de llegar a conclusiones equivocadas prematuramente y nunca encontrar la politica optima. Este tipo de politicas se conocen como $\epsilon$-greedy.

## Implementacion en Julia de la funcion de valor Q


<span style="color: blue">**Advertencia!**  **Hasta nuevo aviso, supondremos que el conjunto de estados es discreto, pues aun no estamos en facultad de desarollar teoria para conjuntos de estados continuos.**.</span>

La forma mas tentadora de almacenar la informacion de $Q$ es utilizando una matriz o un vector. Pero usualmente es mucho mas conveniente usar *tablas hash*, que en Julia (y Python) son llamadas *diccionarios*. Una tabla hash es una estructura de datos que consiste de *claves* y *valores*, asignando a cada clave un valor. Es una estructura no ordenada, a diferencia de un vector. Estas son algunas ventajas

* Usualmente iniciamos los diccionarios vacios y agregamos claves la primera vez que ocurren. En problemas de reinforcement learning hay muchisimos estados que nunca son visitados y que resulta ineficiente o imposible almacenar un valor para cada estado (como en el ajedrez). Las tablas has permite solo asignar valores a las tuplas (estado, accion) observadas, mas similar a como aprendemos en la realidad.
* Con un diccionario los estados pueden ser claves aunque en si mismos sean arrays o objetos compuestos. Muchas otras estructuras necesitan de "nombres" en los estados, y por lo tanto solo pueden ser strings.

Para inicializar un diccionaro vacio en Julia usamos la funcion `Dict`

In [10]:
Q = Dict()

Dict{Any,Any} with 0 entries

Las parejas estado-accion se pueden modelar como tuplas, representadas en Julia con parentesis $(s,a)$. El siguiente ejemplo muestra como manipular $Q$ como un diccionario que tiene como claves las tuplas $(s,a)$. Se pueden usar brackets [ ] para acceder el valor de las claves del diccionario y para signar nuevos valores.

In [11]:
estado = "Some state"
accion = "Some action"
Q[(estado, accion)] = 1.
Q[(estado, accion)] 

1.0

Para acceder a las claves actualmente guardadas se puede usar el comando

In [12]:
keys(Q)

Base.KeyIterator for a Dict{Any,Any} with 1 entry. Keys:
  ("Some state","Some action")

Tambien podemos consultar si el diccionario tiene una clave

In [13]:
haskey(Q, (estado, accion))

true

Una funcion implementada comunmente es las tablas Hash es el metodo `get` que busca una clave y si no la encuentra regresa un valor *default*

In [14]:
default_value = 0.
get(Q, "I am not a key", default_value)

0.0

## Implementacion en Julia de una politica $\epsilon$-greedy en Julia

La implementacion de la politica de un agente puede ser implica o explicita
* Es *explicita* si dado un estado devuelve la distribucion de probabilidad de la politica
* Es *implicita* si dado un estado devuelve una observacion de la distribucion de la politica
Aunque matematicamente modelamos una politica como una distribucion de probabilidad para cada estado. En la practica es muchas veces conveniente modelarla implicitamente. Una de las ventajas de los metodos de funciones de valor es que funcionan muy bien con politicas implicitas, que no necesitan almacenar una gran cantidad de valores por cada combinacion de estado accion, si no que reciben directamente a la funcion de valor $Q$ como argumento una accion.

La politica debe consistir en los siguientes ingredientes:

> ### Ingredientes de una politica de tipo $\epsilon$-greedy:
> * **Inputs**
>    1. `state`: el estado actual en el cual se tiene que tomar una accion
>    2. `actions`: las acciones disponibles al estado `state`
>    3. `Q`: la mejor estimacion hasta el momento de la matriz `Q`, que se usara para tomar la decision greedy $$\mathrm{greedy}(\text{state}) = \mathrm{argmax}_{a\in\text{actions}} Q(\text{state}, a)$$
>    4. $\epsilon$: la probabilidad de tomar una decision completamente al azar y no greedy
> * **Output**
>    + La accion $\mathrm{policy}(\text{state})$ decidida por 
>        - aleatoriamente entre cualquier accion con probabilidad $\epsilon$ 
>        - aleatoriamente entre las acciones que maximicen $Q(\text{state}, \cdot)$ con probabilidad $1-\epsilon$.
        

In [15]:
"""
Returns an action according to an ϵ-greedy policy given an estimate of the action value function Q.
"""
function policy(state::Any, action_space::Any, Q::Dict, ϵ::Float64)
    random_number = rand()
    if random_number < ϵ
        return rand(action_space) #  totalmente aletorio
    else
        max_Q = maximum([get(Q, (state, a), 0.) for a in action_space]) # max_a Q(state, a)
        greedy_actions = [a for a in action_space if get(Q, (state, a), 0.) == max_Q] # argmax_a Q(state, a)
        return rand(greedy_actions) # aleatorio entre las mejores acciones solamente
    end
end

policy

**EJEMPLO DE USO**

In [16]:
Q = Dict()
ϵ = 0.25 # 25% de aleatoriedad total
state = "Some state"
action_space = 0:3
Q[(state, 0)] = Q[(state, 3)] = 1. # Las mejores acciones son 0 y 3
Q[(state, 1)] = Q[(state, 2)] = 0. # Las peores acciones son 1 y 2
for i in 1:30 # Imprime 30 simulaciones de acciones
    print(policy(state, action_space, Q, 0.1), " ")
end

3 0 3 3 0 0 0 0 0 3 3 0 3 0 0 3 1 0 3 0 3 0 0 3 0 3 0 3 3 0 

### <span style='color: red'> Ejercicio 2 </span>

1. Para el juego de MountainCar describe en una o dos oraciones lo que consideres es una politica optima
2. Para el juego de Pong describe en una o dos oraciones que consideres es una politica optima
3. Describe  en una o dos oraciones los potenciales peligros de tener una politica $\epsilon$-greedy en la que $\epsilon$ sea demasiado baja.
4. Resume en una o dos oraciones las diferencias entre las dos ecuaciones de Bellman.
5. Porque decimos que las ecuaciones de Bellman son recursivas?

### <span style='color: red'> Respuestas ejercicio 2 </span>

1. Para el juego de MountainCar describe en una o dos oraciones lo que consideres es una politica optima, 

    R. Sabemos que el Reward es -1 para cada paso en el tiempo, hasta alcanzar la posición objetivo de 0,5
    y el conjunto de acciones y el espacio de acciones es A={[0,push left],[1,no push],[2, push right]}. La política óptima que se puede plantear sería por ejemplo que se incrementara el reward cuando el carro alcance la parte superior de la montaña, al tomar la acción de empujar hacia adelante,considerando los menos pasos posibles.


2. Para el juego de Pong describe en una o dos oraciones que consideres es una politica optima

R.  Sabemos que el espacio de estados es una caja de dimensiones 210x160x3; el conjuno de acciones en este caso es discreto, solo puede obtener los valores 0−5.  Una política óptima podría ser aquella que en que la distancia entre pixeles sea pequeña. La idea es que la política pueda captar que la distancia de la pelotita con el pile sea mínima y optener mayor recompenza.


3. Describe en una o dos oraciones los potenciales peligros de tener una política ϵϵ-greedy en la que ϵϵ sea demasiado baja.

R. Que un agente no explore y que no se pueda obtener una política óptima. Mientras más pequeña sea ϵϵ puede que no se explore todo el espacio. 

4. Resume en una o dos oraciones las diferencias entre las dos ecuaciones de Bellman.

R. La primera ecuación se cumple para todas las políticas, mientras que la segunda solo se cumple para aquella 
que es una política óptima.

5. ¿Por qué decimos que las ecuaciones de Bellman son recursivas?

Debido a que las variables que queremos encontrar dependen del tiempo y la solucion en un momento $t$ se puede 
expresar en términos de la solución al tiempo $t-1$. La idea central del trabajo de Bellman consiste en que para 
encontrar las decisiones óptimas $A_t, A_{t+1},...$ de forma que maximicen el total de pagos recibidos en el tiempo 
$R_{t+1},R_{t+2}, ...$ tenemos que suponer que el estado futuro del sistema es independiente del pasado (pérdida de 
memoria) condicionado a su estado actual y de las decisiones del agente. Esta propiedad se conoce como la propiedad 
de Markov.

En otras palabras, las ecuaciones de Bellman son recursivas en términos de la función de valor estado accion $Q$. La propiedad de Marcov garantiza que en cualquier momento del tiempo el futuro del sistema depende de su estado actual y de la decisión del agente.

## 3. Metodos tabulares de control optimo: Q-Learning y Sarsa

Como mencionamos antes, los metodos de control optimo de Reinforcement Learning que vamos a usar se basan en la estimacion de la funcion de valor $Q$ y para eso usan las Ecuaciones de Bellman y la Ley de los Grandes Numeros (LGN). Vamos a tratar de clarificar un poco mas la razon.

***Ley de los Grandes Numeros:* ** si se tiene una muestra de variables aleatorias independientes e identicas $X_1,...,X_n$ (con esperenza finita) entonces 
$$
\frac{1}{n}\sum_{i=1}^nX_i \stackrel{n\to\infty}{\to} E[X_i].
$$

Las exposiciones formales de los metodos de RL por funciones de valor y sus propiedades de convergencia puede ser muy extensa, pero la intuicion es increiblemente simple. Por ejemplo, para utilizar la LGN junto con la primera ecuacion de Bellman podemos hacer lo siguiente:

<blockquote>
<h3> Derivacion intuitiva de SARSA </h3>
<oi>
<li> Denotemos $Q_t$ la estimacion de $Q$ disponible al tiempo $t$. 
<li> Como 
$$E \left[ R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) \right] - Q(S_t, A_t) = 0$$
segun la primera ecuacion de Bellman, entonces las diferencia 
$$
\delta_t(S_t, A_t) = R_{t+1} + \gamma Q_t(S_{t+1}, A_{t+1})  - Q_t(S_t, A_t)
$$
nos da informacion de que tan bien o que tan mal esta la estimacion actual $Q_t(S_t, A_t)$.
<li>  Como las cantidades $\delta_t(S_t, A_t)$ son "en promedio" cero segun la LGN, entonces podemos dar pequeños pasas en la direccion $\delta_t(S_t, A_t)$ para mejorar nuestra estimacion. Conforme los estados $(S_t, A_t)$ sigan aparenciendo en el futuro, tendremos una buena estimacion de $Q(S_t, A_t)$. Matematicamente
$$
Q_t(S_t, A_t) + \alpha \delta_t \stackrel{t\to \infty}{ \to} Q(S_t, A_t).
$$
<li> La constante $0 < \alpha < 1$ se conoce como **velocidad de aprendizaje**. Valores tipicos son $\alpha \approx 0.025, 0.05, 0.1$ pero puede variar mucho de aplicacion en aplicacion.
<li>  Una interpretacion intuitiva de $\alpha$ es que define una nueva estimacion $Q_{t+1}(S_t, A_t)$, dandole un peso $(1-\alpha)$ a la estimacion actual $Q_t(S_t, A_t)$ y un peso $\alpha$ a la estimacion $R_{t+1} + \gamma Q_t(S_{t+1}, A_{t+1}) $.
</oi>
</blockquote>

La idea anterior se convierte en un metodo de control para encontrar una politica "optima" (ver el ejercicio debajo) llamado SARSA. El nombre SARSA se deriva de la regla de actualizacion de $Q$ que utiliza las variables $S_t, A_t, R_{t+1}, S_{t+1}, A_{t+1}$. La convergencia de este metodo (y tambien los casos en los que podria fallar) esta explicada por las teoria de *Aproximacion Estocastica*.


> ### SARSA 
>
> **Inputs**: 
  * Velocidad de aprendizaje $0 < \alpha < 1$
  * Tasa de descuento de la utilidad en el futuro $0 < \gamma \leq 1$ 
  * Una politica, tipicamente $\epsilon$-greedy.
>  
> **Outputs**
> * Una estimacion de la funcion de valor $Q$ bajo la politica $\epsilon$-greedy
>
> **Algoritmo**
> * **Inicializar** $Q$.
> * **Para cada episodio** con estado-accion inicial $(s_0, a_0)$:
>    + Para cada $t$ mientras el episodio no haya terminado:
>        - **Hacer una transicion** a un nuevo estado-accion $(s_{t+1}, a_{t+1})$ obteniendo un pago $r_{t+1}$.
>        - <span style='color: blue'>**Calcular la diferencia en utilidad segun la regla SARSA**</span>
>          $$ \delta_t = r_{t+1} + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t,a_t) $$
>        - **Actualizar el valor de $Q$**:
>            $$ Q(s,a) = Q(s,a) + \alpha\delta_t$$  
 
Cada vez que el algoritmo encuentre una pareja $(s,a)$ le debe asignar un valor por *default* a $Q(s,a)$. Lo tradicional es asignarle el valor $0$. Las propiedades de convergencia estocastico hacen que no importe este valor inicial.

SARSA es un metodo **tabular** porque Q se calcular para cada posible tupla estado-accion. En la proxima tarea veran metodos aproximados que reducen el espacio de estados a un numero selecto de *features*.






### <span style='color: red'> Ejercicio 3 </span>

1. Con SARSA se obtiene un estimacion de la funcion de valor $Q$ de la politica optima? Si no se obtiene la politica optima, que politica se obtiene? Es necesariamente malo una politica suboptima durante el proceso de aprendizaje?
2. **(Q-LEARNING)** El famoso algoritmo Q-Learning (Watkins 1989) es identico al de SARSA salvo en una linea, reemplazamos la regla SARSA para $\delta_t$ por
$$ \delta_t = r_{t+1} + \gamma \max_a Q_{\pi^*}(s_{t+1}, a) - Q(s_t,a_t) $$
Imitando la derivacion intuitiva de SARSA, explica intuitivamente el algoritmo Q-Learning usando la ecuacion de optimalidad de Bellman.
3. SARSA se clasifica como un metodo de control *on-policy* y Q-Learning como *off-policy*. La razon es que no estiman la misma Q: uno de los metodos estima la optima y el otro no, incluso si ambos se implementan usando la misma politica $\epsilon$-greedy. Investiga en maximo un parrafo mas sobre la diferencia entre SARSA y Q-Learning (o entre control *on-policy* y *off-policy*) y contesta: Es mejor un metodo off-policy que un metodo on-policy?
4. Hay diferencia entre SARSA y Q-Learning conforme $\epsilon \to 0$?

### <span style='color: red'>Respuestas Ejercicio 3 </span>

1. Con SARSA se obtiene un estimacion de la funcion de valor $Q$ de la politica optima? Si no se obtiene la politica optima, que politica se obtiene? Es necesariamente malo una politica suboptima durante el proceso de aprendizaje?

R. No, SARSA compara el estado actual vs el estato actual siguiente.Se optine la mejor política dado epsilon. No es malo, pero usarla depende del problema del que estemos hablando.


	
2. **(Q-LEARNING)** El famoso algoritmo Q-Learning (Watkins 1989) es identico al de SARSA salvo en una linea, reemplazamos la regla SARSA para $\delta_t$ por
$$ \delta_t = r_{t+1} + \gamma \max_a Q_{\pi^*}(s_{t+1}, a) - Q(s_t,a_t) $$
Imitando la derivacion intuitiva de SARSA, explica intuitivamente el algoritmo Q-Learning usando la ecuacion de optimalidad de Bellman.

<blockquote>
<h3> Derivacion intuitiva de Q-Learning </h3>
<oi>
<li> Denotemos $Q_t$ la estimacion de $Q$ disponible al tiempo $t$. 
<li> Como 
$$E \left[ R_{t+1} + \gamma \max_a Q_{\pi^*}(S_{t+1}, A)) \right] - Q(S_t, A_t) = 0$$
segun la segunda ecuacion de Bellman, entonces las diferencia 
$$
\delta_t(S_t, A_t) = R_{t+1} + \gamma max(Q_t(S_{t+1}, A))  - Q_t(S_t, A_t)
$$
nos da informacion de que tan bien o que tan mal esta la estimacion actual $Q_t(S_t, A_t)$ de $Q_*$
<li>  Como las cantidades $\delta_t(S_t, A_t)$ son "en promedio" cero segun la LGN, entonces podemos dar pequeños pasas en la direccion $\delta_t(S_t, A_t)$ para mejorar nuestra estimacion. Conforme los estados $(S_t, A_t)$ sigan aparenciendo en el futuro, tendremos una buena estimacion de $Q(S_t, A_t)$. Matematicamente
$$
Q_t(S_t, A_t) + \alpha \delta_t \stackrel{t\to \infty}{ \to} Q_*(S_t, A_t).
$$

</oi>
</blockquote>


3. SARSA se clasifica como un metodo de control on-policy y Q-Learning como off-policy. La razon es que no estiman la misma Q: uno de los metodos estima la optima y el otro no, incluso si ambos se implementan usando la misma politica ϵϵ-greedy. Investiga en maximo un parrafo mas sobre la diferencia entre SARSA y Q-Learning (o entre control on-policy y off-policy) y contesta: Es mejor un metodo off-policy que un metodo on-policy?


La mayor diferencia entre SARSA y Q-Learning, es que la máxima recompensa para el siguiente estado, no necesariemente es usada para actualizar el valor de Q. En vez de ello, una nueva acción y su recompensa, es seleccinada usando la misma política que fue determinada en la acción original. Q-learning primero actualiza Q y selecciona la siguiente acción basada en la actualización de la política (la óptima). Mientras que SARSA elige la acción y despues actualza Q. 

No creo que un método sea mejor que otro, depende del caso que se desea aprender, en el ejemplo del agente en el acantilado, Q-learning aprende el camino más rápido hacia la salida, sin preocuparse por los riesgos de caerse al acantilado. SARSA es como una persona escrupulosa que evita las zonas peligrosas que están cerca del acantilado tanto como sea posible.


4. Hay diferencia entre SARSA y Q-Learning conforme $\epsilon \to 0$?

No, si epsilon gradualmente tiende a cero. Ambos métodos asintóticamente tiende a la política óptima



## 4. Let's play

Vamos a implementar el algoritmo SARSA para resolver el problema del coche en la colina. Cual es el primer problema que enfrentaremos? Preprocesar los estados.

### Preprocesamiento de estados

Este es el paso que en practicamente ninguna aplicacion de RL podemos escapar. Ademas, los distintos retos dependen del tipo de problema. Algunos puntos comunes que puede surgir son
* **El espacio de estados es demasiado grande?**. Un ejemplo tipico es querer aprender juegos de Atari a traves de observar pixeles, pues muchos pixeles pueden hacer el conjunto de estados inmanejable. En la proxima tarea veremos metodos aproximados que ayudan a resolver el problema de un conjunto de estados muy grande creando features. En metodos tabulares hay que reducir el tamano del conjunto de estados, por ejemplo, en el caso de imagenes de Atari, comprimiendo el tamano de las imagenes y solo guardando la informacion mas importante de los pixeles.
* **El espacio de estados es continuo?**. Si vamos a usar un metodo tabular como SARSA o Q-Learning sera necesario discretizar en un numero finito de valores. Debajo hacemos una demostracion. Nuevamente, los metodos aproximados pueden evitar la necesidad de discretizar.
* **El espacio de estados es markoviano?**. Checar esta condicion es vital y usualmente ignorada. El funcionamiento correcto de los metodos depende del supuesto de Markov, i.e., que el futuro rendimiento dependa unicamente del estado actual y de la decision tomada. De manera practica podemos preguntar: conocer el ultimo estado es suficiente para tomar una decision optima?

La siguiente funcion hace una discretizacion simple de un conjunto de estados de dimension $k$, seleccionando el valor minomo, maximo y el numero de puntos en cada dimension, asumiendo una discretizacion uniforme. Se pueden hacer funciones de discretizacion mucho mas sofisticadas.

In [17]:
"""
For each index `k` of the array `state`, it creates an even grid of length npoints[`k`] 
going from `lower[k]` to `upper[k]`, and returns the closest value of `state[k]` on that grid. 
All the arguments must be of the same length.
"""
function discretize(state::Array, lower::Array, upper::Array, npoints::Array) 
    discretized = Array{Float32}(length(state)) # preallocating is faster (but not necessary)
    for k in 1:length(state)
        grid = linspace(lower[k], upper[k], npoints[k]) # even space
        discretized[k] = grid[indmin(abs(grid - state[k]))] # closest point
    end
    return discretized
end
;

**EJEMPLO DE USO**

In [18]:
function preprocess(state::Array)
    lower = [-1.2, -0.07]
    upper = [0.6, 0.07]
    npoints = [16, 8]
    discretize(state, lower, upper, npoints)
end

mountain_car = gym.make("MountainCar-v0")
state = mountain_car[:reset]()
println(state, " becomes ", preprocess(state))

[-

[2017-06-02 22:00:41,411] Making new env: MountainCar-v0


0.485954,0.0] becomes Float32[-0.48,-0.01]


### Funcion de entrenamiento

Necesitamos una funcion que reciba una estimacion actual de la tabla $Q$ y que devuelva una estimacion mejorada. Notemos que la funcion depende del preprocesamiento que definimos anteriormente.

**APRENDIENDO UN EPISODIO**

In [19]:
"""
The following function plays a gym_episode and learns using SARSA. It receives a current estimate of Q and 
returns an improved estimate Q, the episode reward and the number of steps.
"""
function learn_episode_with_sarsa(
        gym_env, # an openai gym environment
        current_Q = Dict() # current estimation of Q
        ; # the rest are keyword argument, not positional arguments
        preprocess_state = x -> x, # state preprocessing, identity by default
        learning_rate = 0.1,
        discount_rate = 0.999,
        exploration_rate = 0.1,
        render = false, # show game visualization (slower when rendering...)
    )
    # Rename variables for formula simplicity following Sutton
    ϵ, α, γ = exploration_rate, learning_rate, discount_rate
    action_space = 0:(gym_env[:action_space][:n] - 1)
    Q = deepcopy(current_Q) # Q will store the improved Q estimate
    # Reset environment
    state = gym_env[:reset]() # regresa el ambiente al comienzo de un episodio
    state = preprocess_state(state)
    render && gym_env[:render]() # si render es true, crea la visualizacion del estado del juego
    # Get initial action
    action = policy(state, action_space, Q, ϵ) # selecionar accion con politica
    !haskey(Q, (state, action)) && (Q[(state, action)] = 0.) # add key if missing
    # Episode iteration information
    done = false # sera true cuando termine el episodio
    ep_reward = 0. # iniciar pago total en el episodio
    ep_steps = 0 # iniciar pasos totales en el episodio
    while !done 
        # Simulate transition
        new_state, reward, done, info = gym_env[:step](action) # simula una transicion
        new_state = preprocess_state(new_state)
        new_action = policy(new_state, action_space, Q, ϵ) # selecionar accion con politica
        !haskey(Q, (new_state, new_action)) && (Q[(new_state, new_action)] = 0.) # add key if missing
        # Compute δ
        δ = reward + γ*Q[(new_state, new_action)]  - Q[(state, action)] 
        # Update Q 
        Q[(state, action)] = Q[(state, action)] + α*δ
        # Change new states to old states for next step
        state = new_state
        action = new_action
        # Save step information and render if needed
        render && gym_env[:render]()
        ep_reward += reward # agrega el pago de la transicion al pago del ep.
        ep_steps += 1 # agrega un paso al total de pasos en el episodio
    end
    render && gym_env[:render](close = true)
    return Q, ep_reward, ep_steps
end

**EJEMPLO DE USO**

In [20]:
Q = Dict()
Q, reward, steps = learn_episode_with_sarsa(mountain_car, Q, preprocess_state = preprocess, render = true)

(Dict{Any,Any}(Pair{Any,Any}((Float32[-0.6,0.01],0),-1.35244),Pair{Any,Any}((Float32[-0.6,-0.01],2),-1.32793),Pair{Any,Any}((Float32[-0.48,0.01],1),-1.94155),Pair{Any,Any}((Float32[-0.6,-0.01],0),-1.30374),Pair{Any,Any}((Float32[-0.48,-0.01],1),-1.90202),Pair{Any,Any}((Float32[-0.6,-0.01],1),-1.45297),Pair{Any,Any}((Float32[-0.48,0.01],0),-1.83468),Pair{Any,Any}((Float32[-0.48,-0.01],0),-1.89826),Pair{Any,Any}((Float32[-0.6,0.01],1),-1.22577),Pair{Any,Any}((Float32[-0.48,0.01],2),-1.85748)…),-200.0,200)

**APRENDIENDO DE MUCHOS EPISODIOS Y REDUCIENDO TASA DE EXPLORACION**

In [21]:
"""
This function will train a gym environment for many episodes using sarsa and a preprocess
function given by the user, defaults to identity.
"""
function learn_episodes_with_sarsa(
        n_episodes, # number of episodes
        gym_env, # an openai gym environment
        current_Q = Dict() # current estimation of Q
        ; # the rest are keyword argument, not positional arguments
        preprocess_state = x -> x, # state preprocessing, identity by default
        learning_rate = 0.1,
        discount_rate = 0.999,
        exploration_rate = 0.1,
        exploration_decay =  1 - log(2) / n_episodes, # se reducira a la mitad al final del episodio
        render = false, # show game visualization (slower when rendering...)
        verbose = true
    )
    Q = deepcopy(current_Q)
    # Iterate episodes
    reward_history = []
    steps_history = []
    for i in 1:n_episodes
        # learn from episode
        Q, reward, steps = learn_episode_with_sarsa(gym_env, Q, 
                                                    learning_rate = learning_rate, 
                                                    discount_rate = discount_rate, 
                                                    exploration_rate = exploration_rate, 
                                                    render = render,
                                                    preprocess_state = preprocess_state)
        verbose && println("Episodio ", i, " con pago ", reward)
        # reduce exploration_rate
        exploration_rate *= exploration_decay
        # save episode information
        push!(reward_history, reward)
        push!(steps_history, steps)
    end
    return Q, reward_history, steps_history
end

In [22]:
Q, reward_history, steps_history = learn_episodes_with_sarsa(
    2000, mountain_car, Dict(), 
    preprocess_state = preprocess,
    learning_rate = 0.05,
    exploration_rate = 0.25,
    exploration_decay = 1 - log(25) / 2000, # will reduce to 0.01 at the end of training
    render = true # be patient, enjoy watching, switch to false for speed
)

Episodio 1 con pago -200.0
Episodio 2 con pago -200.0
Episodio 3 con pago -200.0
Episodio 4 con pago -200.0
Episodio 5 con pago -200.0
Episodio 6 con pago -200.0
Episodio 7 con pago -200.0
Episodio 8 con pago -200.0
Episodio 9 con pago -200.0
Episodio 10 con pago -200.0
Episodio 11 con pago -200.0
Episodio 12 con pago -200.0
Episodio 13 con pago -200.0
Episodio 14 con pago -200.0
Episodio 15 con pago -200.0
Episodio 16 con pago -200.0
Episodio 17 con pago -200.0
Episodio 18 con pago -200.0
Episodio 19 con pago -200.0
Episodio 20 con pago -200.0
Episodio 21 con pago -200.0
Episodio 22 con pago -200.0
Episodio 23 con pago -200.0
Episodio 24 con pago -200.0
Episodio 25 con pago -200.0
Episodio 26 con pago -200.0
Episodio 27 con pago -200.0
Episodio 28 con pago -200.0
Episodio 29 con pago -200.0
Episodio 30 con pago -200.0
Episodio 31 con pago -200.0
Episodio 32 con pago -200.0
Episodio 33 con pago -200.0
Episodio 34 con pago -200.0
Episodio 35 con pago -200.0
Episodio 36 con pago -200.0
E

(Dict{Any,Any}(Pair{Any,Any}((Float32[-0.24,-0.05],0),-53.792),Pair{Any,Any}((Float32[-0.48,-0.07],0),-1.71042),Pair{Any,Any}((Float32[0.36,0.05],2),-2.24177),Pair{Any,Any}((Float32[0.0,0.03],2),-45.4146),Pair{Any,Any}((Float32[-0.96,0.03],0),-53.9319),Pair{Any,Any}((Float32[-1.08,-0.01],0),-59.8119),Pair{Any,Any}((Float32[-0.96,0.03],2),-44.2811),Pair{Any,Any}((Float32[-0.36,0.05],1),-46.0131),Pair{Any,Any}((Float32[0.0,0.05],2),-27.7692),Pair{Any,Any}((Float32[-0.24,-0.03],1),-65.3071)…),Any[-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0  …  -172.0,-163.0,-168.0,-160.0,-170.0,-155.0,-200.0,-156.0,-163.0,-159.0],Any[200,200,200,200,200,200,200,200,200,200  …  172,163,168,160,170,155,200,156,163,159])

**EJEMPLO DE USO**

In [23]:
Pkg.add("Plots")
#Pkg.add("GR")
#Pkg.update()
using Plots; gr()
plot_every = 100
roll_mean = []
for i in 1:50
   push!(roll_mean, mean(reward_history[(plot_every*(i-1)+1):(plot_every*i)])) 
end
plot(roll_mean, title = "Retorno promedio en batches de 100 episodios")

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of Plots
INFO: Use `Pkg.update()` to get the latest versions of your packages


LoadError: BoundsError: attempt to access 2000-element Array{Any,1} at index [2001:2100]

### <span style='color: red'> Ejercicio 4 </span>

1. Replica el codigo anterior de entrenamiento en 5000 episodios haciendo los cambios necesarios para aprender con Q-Learning en vez de SARSA.

In [24]:
"""
The following function plays a gym_episode and learns using Q-Learning. It receives a current estimate of Q and 
returns an improved estimate Q, the episode reward and the number of steps.
"""
function learn_episode_with_qlearning(
        gym_env, # an openai gym environment
        current_Q = Dict() # current estimation of Q
        ; # the rest are keyword argument, not positional arguments
        preprocess_state = x -> x, # state preprocessing, identity by default
        learning_rate = 0.1,
        discount_rate = 0.999,
        exploration_rate = 0.1,
        render = false, # show game visualization (slower when rendering...)
    )
    # Rename variables for formula simplicity following Sutton
    ϵ, α, γ = exploration_rate, learning_rate, discount_rate
    action_space = 0:(gym_env[:action_space][:n] - 1)
    Q = deepcopy(current_Q) # Q will store the improved Q estimate
    # Reset environment
    state = gym_env[:reset]() # regresa el ambiente al comienzo de un episodio
    state = preprocess_state(state)
    render && gym_env[:render]() # si render es true, crea la visualizacion del estado del juego
    # Get initial action
    action = policy(state, action_space, Q, ϵ) # selecionar accion con politica
    !haskey(Q, (state, action)) && (Q[(state, action)] = 0.) # add key if missing
    # Episode iteration information
    done = false # sera true cuando termine el episodio
    ep_reward = 0. # iniciar pago total en el episodio
    ep_steps = 0 # iniciar pasos totales en el episodio
    while !done 
        # Simulate transition
        new_state, reward, done, info = gym_env[:step](action) # simula una transicion
        new_state = preprocess_state(new_state)
        new_action = policy(new_state, action_space, Q, ϵ) # selecionar accion con politica
        !haskey(Q, (new_state, new_action)) && (Q[(new_state, new_action)] = 0.) # add key if missing
        # Compute δ
        max_Q = maximum([get(Q,(new_state,new_action),0.) for a in action_space])#Maximizo y obtengo la política optima
        δ = reward + γ*max_Q  - Q[(state, action)] 
        # Update Q 
        Q[(state, action)] = Q[(state, action)] + α*δ
        # Change new states to old states for next step
        state = new_state
        action = new_action
        # Save step information and render if needed
        render && gym_env[:render]()
        ep_reward += reward # agrega el pago de la transicion al pago del ep.
        ep_steps += 1 # agrega un paso al total de pasos en el episodio
    end
    render && gym_env[:render](close = true)
    return Q, ep_reward, ep_steps
end

In [25]:
function learn_episodes_with_qlearning(
        n_episodes, # number of episodes
        gym_env, # an openai gym environment
        current_Q = Dict() # current estimation of Q
        ; # the rest are keyword argument, not positional arguments
        preprocess_state = x -> x, # state preprocessing, identity by default
        learning_rate = 0.1,
        discount_rate = 0.999,
        exploration_rate = 0.1,
        exploration_decay =  1 - log(2) / n_episodes, # se reducira a la mitad al final del episodio
        render = false, # show game visualization (slower when rendering...)
        verbose = true
    )
    Q = deepcopy(current_Q)
    # Iterate episodes
    reward_history = []
    steps_history = []
    for i in 1:n_episodes
        # learn from episode
        Q, reward, steps = learn_episode_with_qlearning(gym_env, Q, 
                                                    learning_rate = learning_rate, 
                                                    discount_rate = discount_rate, 
                                                    exploration_rate = exploration_rate, 
                                                    render = render,
                                                    preprocess_state = preprocess_state)
        verbose && println("Episodio ", i, " con pago ", reward)
        # reduce exploration_rate
        exploration_rate *= exploration_decay
        # save episode information
        push!(reward_history, reward)
        push!(steps_history, steps)
    end
    return Q, reward_history, steps_history
end

In [26]:
Q = Dict()
Q, reward, steps = learn_episode_with_qlearning(mountain_car, Q, preprocess_state = preprocess, render = true)

(Dict{Any,Any}(Pair{Any,Any}((Float32[-0.6,0.01],0),-1.39414),Pair{Any,Any}((Float32[-0.36,-0.01],1),-0.399939),Pair{Any,Any}((Float32[-0.72,-0.01],0),-0.1),Pair{Any,Any}((Float32[-0.72,0.01],1),-0.19),Pair{Any,Any}((Float32[-0.72,0.01],0),-0.1),Pair{Any,Any}((Float32[-0.6,-0.01],2),-1.48362),Pair{Any,Any}((Float32[-0.48,0.01],1),-0.972331),Pair{Any,Any}((Float32[-0.6,-0.01],0),-1.62645),Pair{Any,Any}((Float32[-0.72,0.01],2),-0.19),Pair{Any,Any}((Float32[-0.48,-0.01],1),-1.28377)…),-200.0,200)

In [ ]:
Q, reward_history, steps_history = learn_episodes_with_qlearning(
    2000, mountain_car, Dict(), 
    preprocess_state = preprocess,
    learning_rate = 0.05,
    exploration_rate = 0.25,
    exploration_decay = 1 - log(25) / 2000, # will reduce to 0.01 at the end of training
    render = true # be patient, enjoy watching, switch to false for speed
)

Episodio 1 con pago -200.0
Episodio 2 con pago -200.0
Episodio 3 con pago -200.0
Episodio 4 con pago -200.0
Episodio 5 con pago -200.0
Episodio 6 con pago -200.0
Episodio 7 con pago -200.0
Episodio 8 con pago -200.0
Episodio 9 con pago -200.0
Episodio 10 con pago -200.0
Episodio 11 con pago -200.0
Episodio 12 con pago -200.0
Episodio 13 con pago -200.0
Episodio 14 con pago -200.0
Episodio 15 con pago -200.0
Episodio 16 con pago -200.0
Episodio 17 con pago -200.0
Episodio 18 con pago -200.0
Episodio 19 con pago -200.0
Episodio 20 con pago -200.0
Episodio 21 con pago -200.0
Episodio 22 con pago -200.0
